<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/8-transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad utilizando Transformers

### Librerías necesarias


In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 7.2 MB/s 
     |████████████████████████████████| 596 kB 84.0 MB/s 
     |████████████████████████████████| 3.3 MB 60.6 MB/s 
     |████████████████████████████████| 61 kB 604 kB/s 
     |████████████████████████████████| 895 kB 93.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import pipeline, AutoTokenizer
import pandas as pd

# Pipelines

En esta actividad vamos a utilizar Transformers para extraer información del texto. En particular vamos a utilizar los pipelines de:


*   Named Entity Recognition
*   Question Answering



Cargamos los modelos pre-entrenados disponibles en HuggingFace

In [3]:
# Transformer NER
nlp_ner = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner",
    tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner', {"use_fast": False})
    )
# Transformer QA
nlp_qa = pipeline('question-answering', 
        model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
        tokenizer=('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es', {"use_fast": False})
    )
# Lo necesitamos para sacar el texto asociado a cada token. La predicción de ner viene con [UNK] cuando no reconoce el
# como un token
tok = AutoTokenizer.from_pretrained('mrm8488/bert-spanish-cased-finetuned-ner')

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/136 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/237k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/237k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
tok

PreTrainedTokenizerFast(name_or_path='mrm8488/bert-spanish-cased-finetuned-ner', vocab_size=31002, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

# Named Entity Recognition
El pipeline NER detecta diferentes entidades en el texto analizado. Ejemplo:

In [5]:
texto = 'Carlos es estudiante de medicina, vive en Santiago hace 3 años'
df_result = pd.DataFrame(nlp_ner(texto))
print(df_result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  entity     score  index      word start   end
0  B-PER  0.998058      1    Carlos  None  None
1  B-LOC  0.999609      9  Santiago  None  None


Vemos que el pipeline detecta:
*   Una entidad de persona en el token índice 1
*   Una entidad de localidad en el token índice 9

Como se puede observar en el resultado anterior, el pipeline también devuelve la palabra asociada al token (columna *word*)

Puede suceder que entidad detectada corresponda a la concatenación de 2 o más tokens. Ejemplo:

In [6]:
texto = 'Carlitos es estudiante de medicina, vive en Santiago hace 3 años'
df_result = pd.DataFrame(nlp_ner(texto))
print(df_result)

  entity     score  index      word start   end
0  B-PER  0.999317      1      Carl  None  None
1  I-PER  0.709062      2    ##itos  None  None
2  B-LOC  0.999640     10  Santiago  None  None


En este caso, la persona *Carlitos* está compuestas por 2 tokens (*Carl* y *##itos*)

# **Pregunta 1**:
¿A qué corresponden los tags B-PER e I-PER? 
https://huggingface.co/dslim/bert-base-NER

En general, corresponden a nombres de personas.
B-PER al principio, e I-PER al interior.


También puede suceder que el string asociado a la persona, no se corresponda con ningún token del tokenizador, pero de todas formas el pipeline detecta la entidad asociada. Ejemplo:

In [7]:
texto = 'Juan es estudiante de medicina, vive en Santiago hace 3 años'
df_result = pd.DataFrame(nlp_ner(texto))
print(df_result)

  entity     score  index      word start   end
0  B-PER  0.998483      1     [UNK]  None  None
1  B-LOC  0.999622      9  Santiago  None  None


Para poder obtener el texto asociado a esa entidad debemos utilizar el mapeo realizado por el tokenizador de la siguiente manera.

In [8]:
# Mapping tiene una lista con índices (start, end) correspondiente al texto de cada token, en el mismo orden
mapping = tok(texto, return_offsets_mapping=True)['offset_mapping']
# Llenamos las columnas start, end y text con el texto correspondiente a cada token
for i, row in df_result.iterrows():
    df_result.loc[i, 'text'] = texto[mapping[row['index']][0]:mapping[row['index']][1]]
    df_result.loc[i, 'start'] = mapping[row['index']][0]
    df_result.loc[i, 'end'] = mapping[row['index']][1]
print(df_result)

  entity     score  index      word start end      text
0  B-PER  0.998483      1     [UNK]     0   4      Juan
1  B-LOC  0.999622      9  Santiago    40  48  Santiago


De esta manera, podemos obtener el comienzo y fin de cada string que fué detectado como una entidad.

# Question Answering
El pipeline QA permite realizar preguntas en un determinado context (texto). en los siguientes ejemplos se muestra modo utlizarlo.

In [9]:
texto = 'Carlos es estudiante de medicina, vive en Santiago hace 3 años'

question = '¿Quién Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
print(respuesta)

{'score': 0.9089308977127075, 'start': 10, 'end': 33, 'answer': 'estudiante de medicina,'}


In [10]:
question = '¿Dónde vive Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
print(respuesta)

{'score': 0.9807462692260742, 'start': 42, 'end': 50, 'answer': 'Santiago'}


# **Pregunta 2**:
Si uno realiza una pregunta que claramente no tiene respuesta en ese contexto, qué hace el modelo y cómo lo podría resolver?

El modelo, al no encontrar ninguna respuesta, responde el ultimo concepto que aparece en el texto.
En vez de eso, mejor deberia decir que no hay una respuesta.

In [11]:
question = '¿De que color es el gato de Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
print(respuesta)

{'score': 1.5464765965589322e-06, 'start': 58, 'end': 61, 'answer': 'años'}


In [12]:
texto = 'Carlos es estudiante de medicina, vive en Santiago hace 3 años seguidos'
question = '¿De que color es el gato de Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
print(respuesta)

{'score': 1.1212192703169421e-06, 'start': 58, 'end': 70, 'answer': 'años seguidos'}


In [13]:
question = '¿De que color es el gato de Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
if respuesta['score'] < 0.01:
  print("No tengo respuesta")
else:
  print(respuesta)

No tengo respuesta


# Aplicación de ejemplo
Se tiene un conjunto de documentos correspondientes a contratos de locación de una inmobiliaria. Para cada contrato, se pretende extraer de manera automática:
* Las personas que intervienen en el contrato
* Determinar quién es el locador y el locatario

In [14]:
contrato1 = """

CONTRATO DE LOCACIÓN DE INMUEBLE CON DESTINO HABITACIONAL
 
 
En la Ciudad de Bariloche, a los 22 días del mes de Octubre de 2021, entre Carlos Gómez con documento DNI N° 123456789, con domicilio en la calle Catamarca 1102, en lo sucesivo denominado como LOCADOR, y por la otra Rubén Gonzales con documento DNI N° 123987654, con domicilio en el inmueble locado, en adelante denominado como LOCATARIO, convienen en celebrar el presente contrato de locación de vivienda, sujeto a las cláusulas siguientes y a las disposiciones del Código Civil y Comercial.

PRIMERA (OBJETO): EL LOCADOR cede en locación al LOCATARIO, que acepta ocupar en tal carácter, el inmueble ubicado en calle Santiago 1234 de la cuidad de Bariloche. El LOCATARIO se obliga a destinar el inmueble locado para vivienda familiar, no pudiendo ello ser modificado, sin el consentimiento expreso del/la  LOCADOR. 

SEGUNDA (PLAZO):  Las partes acuerdan que el plazo de vigencia de la locación será de 36 (TREINTA Y SEIS) meses (ESTE ES UN PLAZO MÍNIMO, PUEDE SER POR MÀS TIEMPO, NUNCA POR MENOS) conforme lo establece Art. 1198 del Código Civil y Comercial, el mismo se inicia con fecha ….. del mes de …………. del año DOS MIL ……….. por lo que su vencimiento se producirá de pleno derecho e indefectiblemente el día…… del año DOS MIL

TERCERA (PRECIO Y AJUSTE): Las partes convienen un canon locativo de PESOS ($__._______) mensuales  para el primer año de contrato (EL PRECIO DEBE SER EL MISMO DURANTE LOS 12 MESES INICIALES). El precio pactado será actualizado anualmente utilizando el Índice para Contratos de Locación (ICL) elaborado y publicado por el Banco Central de la República Argentina (BCRA), conforme lo estipula el Artículo 14 de la ley Nº 27551. El nuevo canon será informado al inquilino con al menos 10 días previos al vencimiento del pago, en la dirección electrónica consignada en el presente Contrato.
CUARTA (FECHA/LUGAR DE PAGO): EL/ LA LOCATARIO/
A se obliga a abonar el alquiler convenido por mes entero y adelantado, entre el 1 y el 10 de cada mes. El pago se efectuará por transferencia electrónica o depósito bancario en la Cuenta N° …………del Banco……….., CUIT __-_________-_, CBU _____________________.
Recibido el Pago, EL/LA/s LOCADOR/A extenderá al LOCATARIO/A la factura electrónica correspondiente, dentro de las 72 hs. (Res. N° 4004-E AFIP), la cual será remitida al LOCATARIO/A a la dirección electrónica consignada en el presente.

QUINTA (MORA): La mora en el pago de los alquileres se producirá de forma automática por el mero transcurso del tiempo y sin necesidad de interpelación ni gestión previa de ninguna naturaleza. Producida la mora los alquileres siempre deberán abonarse con una multa equivalente a la tasa activa por plazo fijo en el Banco de la Nación Argentina, durante el tiempo que demore en efectivizar el pago de los alquileres adeudados. ----------------------------------------------------------

SEXTA (INTRANSFERIBILIDAD): El presente contrato de locación es intransferible. Le queda prohibido al LOCATARIO/A cederlo o subarrendarlo total o parcialmente, sin consentimiento del/la LOCADOR/A, ya sea en forma gratuita u onerosa, ni se podrá dar el inmueble en préstamo de uso. Asimismo, queda prohibido usarlo indebidamente por el/la LOCATARIO/A contrariando las leyes, ni darle otro destino que el establecido de vivienda personal únicamente..------------------------------------------------------------------------

SÉPTIMA (LA CLÁUSULA DE GARANTÍA): De conformidad con el Art 13 de la Ley 27551, se presenta
A) Título de propiedad inmueble;
b) Aval Bancario;
c) Seguro de Caución; (Conforme Resolución 376/202 de la Superintendencia de Seguros de la Nación (“SSN”))
d) Garantía de fianza; o fiador solidario; o
e) Garantía personal del locatario, que se documenta con recibo de sueldo, certificado de ingresos o cualquier otro medio fehaciente. En caso de ser más de un locatario, deben sumarse los ingresos de cada uno de ellos a los efectos de este artículo.
El locador no puede requerir una garantía que supere el equivalente a cinco (5) veces el valor mensual de la locación, salvo que se tratase del supuesto previsto en el inciso e) en el cual podrá elevarse dicho valor hasta un máximo de diez (10) veces. Bajo tales condiciones, el locador debe aceptar una de las garantías propuestas por el locatario.

OCTAVA (OBLIGACIONES DE LAS PARTES): Son obligaciones del LOCATARIO/A:
a) El pago de los siguientes servicios: I) electricidad, gas, agua potable, II) cargas y contribuciones asociadas al destino de vivienda del inmueble, III) las expensas que deriven de gastos habituales ordinarios, entendiéndose por tales aquellos que se vinculan a los servicios normales y permanentes a disposición del LOCATORIO/A, independientemente de que sean considerados como expensas comunes ordinarias o extraordinarias.
b) Transferir la titularidad de los servicios de energía eléctrica, agua, teléfono, gas, TV x cable e internet, a su nombre y cargo en un plazo no superior a los sesenta (60) días corridos, a partir de la suscripción del presente, y abonar su suministro hasta la fecha de corte de los servicios al momento de entrega del inmueble al LOCADOR/A, en el que deberá presentar la correspondiente baja. Para el caso de registrarse deuda previa a este contrato en alguno de estos servicios, EL/LA LOCATARIO/A podrá cancelarla y compensar lo invertido de próximos alquileres. Esta decisión deberá anticipársela por vía electrónica al LOCADOR/A (Art. 1204 bis. del Código Civil y Comercial). 
c) Respetar la normativa local y exigencias de cualquier otra jurisdicción o naturaleza, por lo que asume la responsabilidad por todas las consecuencias que deriven de su inobservancia.
d) Abonar las costas judiciales y extrajudiciales que se originen en el incumplimiento de sus obligaciones contractuales.
Son obligaciones del LOCADOR/A:
a) Realizar todas las reparaciones que requiera el inmueble, así como las necesarias para el normal funcionamiento de los servicios públicos, con excepción de aquellos arreglos cuya necesidad surjan por la responsabilidad del LOCATARIO/A, excluyendo en este caso la que se produzca por el normal uso de la cosa.
b) Notificar al LOCATARIO/A, con al menos 72 hs de antelación, cualquier visita que quiera realizar sobre el inmueble.
c) Abonar las expensas comunes extraordinarias (gastos no habituales) durante la relación locativa.
d) Abonar las cargas y contribuciones que graven el inmueble (impuesto inmobiliario), y cualquier otro que grave a la propiedad.
e) Abonar las costas judiciales y extrajudiciales que se originen en el incumplimiento de sus obligaciones contractuales.

NOVENA (REPARACIONES Y DEVOLUCIÓN): En caso de negativa o silencio del LOCADOR/A ante un reclamo del LOCATARIO/A debidamente notificado, para que efectúe alguna reparación urgente, el LOCATARIO/A puede realizarla por sí, con cargo al locador, una vez transcurridas al menos veinticuatro (24) horas corridas, contadas a partir de la recepción de la notificación.
Si las reparaciones no fueran urgentes, el locatario debe intimar al locador para que realice las mismas dentro de un plazo que no podrá ser inferior a diez (10) días corridos, contados a partir de la recepción de la intimación, cumplido el cual podrá proceder en la forma indicada en el párrafo precedente.
Estos gastos deberán estar debidamente acreditados, pudiendo el/la LOCATARIO/A en estos casos descontarlos del importe del alquiler.

DÉCIMA (DEPÓSITO): En garantía de las obligaciones contraídas mediante este contrato, el/la LOCATARIO/A da en depósito al LOCADOR/A la suma equivalente al valor de un mes del precio de alquiler inicial, correspondiente al valor del primer mes del contrato. En el momento mismo de restitución del inmueble EL/ LA LOCADOR/A deberá devolver al LOCATARIO/A el depósito en garantía, actualizado su valor al del último mes del contrato (Art. 1196, Código Civil y Comercial)
Para el caso que EL/LA LOCATARIO/A no presente libre deuda o exista/n servicio/s público/s domiciliario/s y/o expensas pendientes de pago o liquidación al momento de la entrega del inmueble, EL/LA LOCADOR/A podrá retener del depósito el valor de la última liquidación del servicio o la expensa pendiente de pago.
 
El/la LOCATARIO/A se obliga a preservar el inmueble y a restituirlo en iguales condiciones a las de recepción, salvo por el normal desgaste propio del tiempo y el uso adecuadode la cosa.

Serán a exclusivo cargo del LOCATARIO/A todas las reparaciones, reconstrucciones y/o refacciones que fueran menester realizar para el debido cumplimiento de esta obligación, cualquiera fuera la causa, naturaleza o cuantía del deterioro, y sin derecho a reembolso alguno a su favor. En caso de no hacerlo, el/la LOCADOR/A tendrá derecho a realizarlo a cuenta del/la LOCATARIO/A.

Serán a exclusivo cargo del/la LOCADOR/A y sin derecho a reembolso alguno a su favor las reparaciones, reconstrucciones y/o refacciones que fueran menester realizar en el inmueble por daños causados por fuerza mayor, hechos de terceros, vicios redhibitorios, o por el normal uso de la cosa . En caso de no hacerlo, el/la LOCATARIO/A tendrá derecho a realizarlo a cuenta del/la LOCADOR/A, pudiendo descontarlo del importe del alquiler.

En todos los casos, ambas partes podrán presentar presupuestos para su comparación y estipular de este modo el costo final de los arreglos.
El LOCATARIO declara conocer y aceptar el reglamento de copropiedad y su falta de cumplimiento será causal de rescisión.
DECIMASEGUNDA (INCUMPLIMIENTO): La violación por parte de EL/LA LOCATARIO/A de cualquiera de las obligaciones que asume en el presente dará derecho a EL/LA LOCADOR/a para optar entre exigir su cabal cumplimiento o dar por resuelto el presente contrato y exigir el inmediato desalojo del inmueble con el pago de los daños y perjuicios pertinentes

DECIMATERCERA (FALTA DE PAGO): La falta de pago de 2 meses de alquiler consecutivos dará derecho al LOCADOR/A a, previo a los trámites establecidos por la ley, considerar irrevocablemente rescindido el presente contrato de locación y convenio, pudiendo pedir el desalojo del bien y con derecho a reclamar las pérdidas e intereses que ocasione el incumplimiento. Previo a ello, en todos los casos el/la LOCADOR/A deberá intimar fehacientemente al LOCATARIO/A el pago de la cantidad debida, otorgando para ello un plazo que nunca debe ser inferior a 10 (DIEZ) días corridos contados a partir de la recepción de la intimación, consignando el lugar de pago

DECIMACUARTA (PRIMER MES): El/LA LOCATARIO/A abona en este acto la cantidad de PESOS ……………………… ($...........-) en concepto del alquiler correspondiente al mes de ………………. de …………... Por este primer canon, EL/ LA LOCADOR/A remitirá al LOCATARIO/A la correspondiente factura electrónica.

DECIMAQUINTA (FINALIZACIÓN/ENTREGA DE LLAVES): Al vencimiento del contrato, el/la LOCATARIO/A deberá devolver las llaves al LOCADOR/A. La recepción por parte del/la LOCADOR/A del inmueble arrendado, aún sin reserva alguna de su parte, no importará conformidad con el estado del inmueble, por lo que el/la LOCATARIO/A deberá requerir comprobante de recepción en el que conste expresamente dicha circunstancia para su liberación

DECIMASEXTA (RESCISIÓN ANTICIPADA): De acuerdo a lo establecido en el Art. 1220 del Código Civil y Comercial, el/la LOCATARIO/A puede resolver el contrato en cualquier momento, sin costo alguno de su parte, si el/la LOCADOR/A incumple la obligación de conservar el inmueble con aptitud para el uso y goce convenido; o por la garantía de evicción o la de vicios redhibitorios. “EL/LA LOCATARIO/A” podrá, transcurridos los seis primeros meses de vigencia de la relación locativa, resolver la contratación, debiendo notificar en forma fehaciente su decisión a “EL/LA LOCADOR/A” con una antelación mínima de treinta días. “EL/LAla LOCATARIO/A” podrá hacer uso de la opción resolutoria en el primer año de vigencia de la relación locativa; deberá abonar a “EL/LA LOCADOR/A“ en concepto de indemnización la suma equivalente a un mes y medio de alquiler al momento de desocupar la vivienda, y la de un solo mes si la opción se ejercita transcurrido dicho lapso. Transcurridos los primeros seis (6) meses de relación, si EL/LA LOCATARIO/A notifica al LOCADOR/A su decisión con una anticipación de tres (3) meses o más, no corresponde el pago de indemnización por rescisión anticipada (Art. 1221 del Código Civil y Comercial )


En prueba de conformidad previa lectura y ratificación se firman tres ejemplares de un mismo tenor y a un solo efecto. Recibiendo en este mismo acto EL/LA/S LOCATARIO/A las llaves y la tenencia del inmueble locado.
En la CIUDAD DE BARILOCHE a los 30días del mes de OCTUBRE del 2021.

"""

In [15]:
contrato2 = """
CONTRATO DE LOCACIÓN DE VIVIENDA

PRIMERA: PARTES. Entre Mauricio Perez (DNI 123456789) por una parte, en adelante el LOCADOR, con domicilio en Ayacuhco 1234, CABA, y Fernando Villejas (DNI 129876543) que en adelante se denominará el LOCATARIO (inquilino/a) con domicilio en Garay 1455, convienen celebrar el presente contrato de locación, el que se regirá por el Código Civil y Comercial de la Nación (CCyCN), leyes aplicables a la materia y las cláusulas de este contrato.

SEGUNDA: OBJETO. EL/LA LOCADOR/A cede en locación a la LOCATARIO/A, que acepta, el inmueble sito en la calle Junin 8764 de la CABA. El inmueble objeto de locación tendrá por destino la vivienda familiar de EL/LA LOCATARIO/A, no pudiendo éste/a modificarlo salvo consentimiento expreso de EL/LA LOCADOR/A (art. 1196, CCyCN).

TERCERA: PLAZO. Las partes convienen que la presente locación se extenderá por TRES AÑOS (plazo mínimo / 36 meses), desde el día __/__/__ hasta el día __/__/__, inclusive (art. 1198, CCyCN).

CUARTA: PRECIO. Por la locación, las partes conviene un canon locativo de PESOS _____________________ ($__._______) por mes para el primer año de contrato (12 meses iniciales).
QUINTA: AJUSTE ANUAL. Para el segundo y tercer año el canon mensual definido en la cláusula anterior se actualizará conforme el art. 14 de la Ley N° 27.551. EL/LA LOCADOR/A realizará el cálculo indexatorio con una anticipación de QUINCE (15) días al vencimiento del primer mes del nuevo año de contrato. El nuevo valor se le informará al LOCATORIO/A por vía electrónica, al menos diez (10) días antes que venza el pago del mes (art. 14, Ley N° 27.551).

SEXTA: PERÍODO Y LUGAR DE PAGO. EL/ LA LOCATARIO/A se obliga a abonar el alquiler convenido por mes entero y adelantado, entre el 1 y el 10 de cada mes. El pago se efectuará por transferencia electrónica o depósito bancario en la Cuenta N° ________________ del Banco _____________, CUIT __-_________-_, CBU _____________________, titular del LOCADOR/A.
En contrapartida, EL/ LA LOCADOR/A extenderá al LOCATARIO/A la factura electrónica correspondiente, dentro de las 72hs (Res. N° 4004-E AFIP).
Para el caso que EL/LA LOCATARIO/A deshabitara el inmueble durante el transcurso del mes, deberá abonará el proporcional a los días en que haya habitado el inmueble.

SEPTIMA. DEMORA: La mora en el pago del alquiler se producirá de forma automática. Por ésta se abonará la tasa activa por plazo fijo del Banco de la Nación Argentina, durante el tiempo que demore en efectivizar el pago de los alquileres adeudados.

OCTAVA: EXPENSAS, SERVICIOS E IMPUESTOS. EL/ LA LOCATARIO/A tiene a su cargo el pago en tiempo y forma de a): servicios de energía eléctrica, agua y gas; b): cargas y contribuciones asociadas al destino de vivienda del inmueble; c) las expensas que deriven de gastos habituales ordinarios, entendiéndose por tales aquellos que se vinculan a los servicios normales y permanentes a disposición del LOCATORIO/A, independientemente de que sean considerados como expensas comunes ordinarias o extraordinarias.
EL/LA LOCADOR/A tiene a su cargo las cargas y contribuciones que graven el inmueble (impuesto inmobiliario) y las expensas comunes extraordinarias (gastos no habituales) (art. 1209, CCyCN).

NOVENA: TITULARIDAD SERVICIOS. EL/ LA LOCATARIO/A, dentro de los TREINTA (30) días de suscripto el presente, transferirá a su nombre los servicios públicos, el servicio de TV por cable e internet.
Para el caso de registrarse deuda previa a este contrato en alguno de estos servicios, EL/LA LOCATARIO/A podrá cancelarla y compensar lo invertido de próximos alquileres. Esta decisión deberá anticipársela por vía electrónica al LOCADOR/A (art. 1204 bis., CCyCN).
EL/LA LOCADOR/A, dentro de los TREINTA (30) días de terminado el contrato, asegurará el cambio de titularidad del total de servicios, desvinculando así al LOCATORIO/A saliente.

DECIMA: REGLAMENTOS Y CONSORCIO. El/ LA LOCATARIO/A se compromete a respetar los reglamentos de Copropiedad y Administración y el Interno del edificio, cuyas copias se hace entrega en el presente acto, siendo responsable ante el consorcio de propietarios de las transgresiones estipuladas en los mismos.
EL/LA LOCADOR/A, por su parte, a través del Anexo que acompaña el presente, extiende autorización al LOCATARIO/A para que participe en su nombre en las asambleas que se desarrollen durante el contrato.
EL/ LA LOCADOR/A podrá retirar esta autorización por asamblea. Esta comunicación se efectuará por vía electrónica, 48 hs. corridas antes del inicio de ésta.

DÉCIMA PRIMERA. MEJORAS Y MODIFICACIONES: EL/LA LOCATARIO/A no podrá hacer modificaciones de ninguna naturaleza en la propiedad, sin consentimiento previo del LOCADOR/A expresado por vía electrónica.
Para el caso que las modificaciones previstas impliquen mejoras del inmueble EL/LA LOCADOR/A deberá reembolsar al LOCATARIO/A lo invertido.

DECIMA SEGUNDA: PROHIBICIÓN. El presente contrato de locación es intransferible, obligando exclusivamente a los aquí firmantes. Queda prohibido al LOCATARIO/A ceder o subarrendar total o parcialmente el inmueble sin consentimiento del/la LOCADOR/A, ya sea en forma gratuita u onerosa.
Asimismo, queda estrictamente prohibido usarlo indebidamente por el/la LOCATARIO/A contrariando las leyes, ni darle otro destino que el establecido de vivienda familiar únicamente.

DECIMA TERCERA: RESPONSABILIDADES. EL/ LA LOCATORIO/A tiene a su cargo la obligación de mantener el inmueble y restituirlo en el estado que lo recibió, excepto por deterioros ocasionados por el mero transcurso del tiempo y por el uso regular (art. 1210 CCyCN).
EL/LA LOCADOR/A, por su parte, entregarlo en los términos previstos, a conservarlo para que sirva al uso y goce convenido y a efectuar las reparaciones que exija el deterioro en su calidad o defecto, originado por cualquier causa no imputable al LOCATORIO/A (art. 1201, CCyCN).

DECIMA CUARTA: REPARACIONES. En caso de negativa o silencio del locador ante un reclamo del LOCATORIO/A debidamente notificado, para que efectúe alguna reparación urgente, EL/LA LOCATORIO/A puede realizarla por sí, con cargo al locador, una vez transcurridas al menos veinticuatro (24) horas corridas, contadas a partir de la recepción de la notificación.
Si las reparaciones no fueran urgentes, EL/LA LOCATORIO/A debe intimar al LOCADOR/A para que realice las mismas dentro de un plazo que no podrá ser inferior a diez (10) días corridos, contados a partir de la recepción de la intimación, cumplido el cual podrá proceder en la forma indicada en el párrafo precedente (art. 1201, CCyCN).

"""

In [16]:
# Dividimos el texto en párrafos
parrafos = contrato2.split('\n')
df = pd.DataFrame(data=parrafos, columns=['texto'])
df.head(10)

,texto
0,
1,CONTRATO DE LOCACIÓN DE VIVIENDA
2,
3,PRIMERA: PARTES. Entre Mauricio Perez (DNI 123...
4,
5,SEGUNDA: OBJETO. EL/LA LOCADOR/A cede en locac...
6,
7,TERCERA: PLAZO. Las partes convienen que la pr...
8,
9,"CUARTA: PRECIO. Por la locación, las partes co..."


In [17]:
# Removemos celdas vacias
df = df[df.texto.str.len() > 2]
df.head()

,texto
1,CONTRATO DE LOCACIÓN DE VIVIENDA
3,PRIMERA: PARTES. Entre Mauricio Perez (DNI 123...
5,SEGUNDA: OBJETO. EL/LA LOCADOR/A cede en locac...
7,TERCERA: PLAZO. Las partes convienen que la pr...
9,"CUARTA: PRECIO. Por la locación, las partes co..."


# **Pregunta 3:**
¿Qué hace la función *get_persons()*?

Imprime los B-PER e I-PER del texto, es decir, encuentra las personas.

In [18]:
def intervals(l):
    res = list()
    start = None
    for i in range(len(l)):
        if l[i] and start is None:
            start = i
        if not l[i] and start is not None:
            end = i - 1
            res.append((start, end))
            start = None
    if start is not None:
        res.append((start, len(l) - 1))
    return res

def get_persons(texto):
  df_result = pd.DataFrame(nlp_ner(texto))
  if df_result.empty:
    return []
  df_result = df_result[df_result.entity.isin(['B-PER', 'I-PER'])].copy()
  # Mapping tiene una lista con índices (start, end) correspondiente al texto de cada token, en el mismo orden
  mapping = tok(texto, return_offsets_mapping=True)['offset_mapping']
  for i, row in df_result.iterrows():
    df_result.loc[i, 'text'] = texto[mapping[row['index']][0]:mapping[row['index']][1]]
    df_result.loc[i, 'start'] = mapping[row['index']][0]
    df_result.loc[i, 'end'] = mapping[row['index']][1]

  # Detalle: los intervalos no incluyen los espacios entre tokens (por eso el +1)
  yy = [False] * len(texto)
  
  for ii, row in df_result.iterrows():
    yy[row.start:row.end + 1] = [True] * (row.end + 1 - row.start)
  nombres = [texto[i[0]:i[1]] for i in intervals(yy)]
  return nombres

# **Pregunta 4:**
Utilice la función *get_person* para detectar las personas en los primeros párrafos de los contratos, y el modelo de QA para determinar su rol en el mismo.

In [19]:
parrafos1 = contrato1.split('\n')
df1 = pd.DataFrame(data=parrafos1, columns=['texto'])
df1 = df1[df1.texto.str.len() > 2]

for text in df1.texto:
  persons = (get_persons(text))
  for person in persons:
    question = "¿Qué rol tiene {}?".format(person)
    respuesta = nlp_qa({'question': question, 'context': text})
    print(question)
    print(respuesta)

¿Qué rol tiene Carlos Gómez?
{'score': 0.438506156206131, 'start': 193, 'end': 201, 'answer': 'LOCADOR,'}
¿Qué rol tiene Rubén Gonzales?
{'score': 0.23914510011672974, 'start': 328, 'end': 338, 'answer': 'LOCATARIO,'}
¿Qué rol tiene SÉPTIMA?
{'score': 0.4055003821849823, 'start': 81, 'end': 83, 'answer': 'se'}
¿Qué rol tiene LO?
{'score': 0.012785117141902447, 'start': 24, 'end': 45, 'answer': 'del LOCATARIO/A todas'}
¿Qué rol tiene A?
{'score': 0.018756823614239693, 'start': 0, 'end': 13, 'answer': 'DECIMATERCERA'}


In [20]:
nombres = get_persons(df1.texto)

ValueError: ignored

In [21]:
for text in df.texto:
  persons = (get_persons(text))
  for person in persons:
    question = "¿Qué rol tiene {}?".format(person)
    respuesta = nlp_qa({'question': question, 'context': text})
    print(question)
    print(respuesta)

¿Qué rol tiene Mauricio Perez?
{'score': 0.37017059326171875, 'start': 84, 'end': 92, 'answer': 'LOCADOR,'}
¿Qué rol tiene Fernando Villejas?
{'score': 0.036881983280181885, 'start': 200, 'end': 209, 'answer': 'LOCATARIO'}
¿Qué rol tiene LOCA?
{'score': 0.00026494244229979813, 'start': 72, 'end': 79, 'answer': 'acepta,'}
¿Qué rol tiene A?
{'score': 0.00018873166118282825, 'start': 55, 'end': 67, 'answer': 'LOCATARIO/A,'}
¿Qué rol tiene CA?
{'score': 0.0008364066015928984, 'start': 126, 'end': 131, 'answer': 'CABA.'}
¿Qué rol tiene A?
{'score': 0.00018873166118282825, 'start': 55, 'end': 67, 'answer': 'LOCATARIO/A,'}
